Chapitre 11 - Jointures SQL et Update
===

Maintenant que nous pouvons faire des insertions, il nous revient de pouvoir faire les actions suivantes :
- lier les auteurs à une publication
- mettre à jour les données d'un objet

Pour commencer, voici une petite idée :

- nous allons rajouter un formulaire de création de données
    - nous ajouterons la possibilité de lier les utilisateurs qui les rédigent à cet objet
- nous allons rajouter un formulaire d'édition

## Commencer par le commencement : la jointure

En SQL, une jointure est une requête effectuée sur plusieurs tables ensemble. Cela permet ainsi d'éviter de multiplier les communications avec le serveur SQL. Cependant, mal faites, elles peuvent rapidement poser des problèmes de performance. 

L'avantage de SQLAlchemy est le suivant : les jointures sont faciles à effectuer. La jointure que nous voulons faire est simple : nos utilisateurs pouvant créer ou éditer des objets, on a une relation de n-n (ou *many-to-many*) entre notre Place et notre User. Dans ce genre de conditions, on crée une table de liaison entre nos objets: c'est la table authorship !

**Note:** La documentation des jointures est disponible à cette adresse : http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html



In [6]:
# On remet en place la configuration Flask / MySQL

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask("Nom")
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///cours-flask/db.sqlite'
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False
db = SQLAlchemy(app)

In [7]:
# Code nécessaire
import datetime


class Authorship(db.Model):
    __tablename__ = "authorship" # nom de ma table dans la base de données sqlite
    authorship_id = db.Column(db.Integer, nullable=True, autoincrement=True, primary_key=True)
    # jointure très simple avec db.ForeignKey
    authorship_place_id = db.Column(db.Integer, db.ForeignKey('place.place_id'))
    authorship_user_id = db.Column(db.Integer, db.ForeignKey('user.user_id'))
    authorship_date = db.Column(db.DateTime, default=datetime.datetime.utcnow)
    # db.relationship lie la classe user à mon champ User: quand je fais x=Authorship.query.get(1)
    # je pourrais avoir accès à Authorship.user_id puis faire u=User.query.get(x.authorship.user_id)
    # ca me fais donc deux requêtes
    # mais avec db.relationship, je fais juste x.user: il instancie automatiquement utilisateur: il 
    # fait la jointure de lui_même et me donne l'objet tel que je l'utilise, donc j'ai accès à x.user.user_nom par ex
    # 
    # backpopulates: permet à sqlalchemy de comprendre la relation entre les objets
    # Authorship a relation vers 1 user et 1 place
    # mais la classe User a n relations vers Authorship
    # User.authorship peut y être 0 à n fois
    # donc je peux avec backpopulates faire une relation en chiasme entre les objets: authorships dans user sera 
    # une liste d'objets authorships, alors que user de authorship ser aune seule instance de user authorships
    # l'intérêt est de pouvoir modifier directement l'objet à travers sa relation
    # 1 seul objet authorship=1 existe que je sois dans Authorship ou dans User.
    # backpopulates instancie une relation entre le user de Authorship et le authorships de User
    # 
    # EXEMPLES
    # Roger.authorships[0].user.user_nom == Roger.user_nom
    # 
    # pour afficher tous les lieux modifiés par Roger
    # for out in Roger.authorships:
    # print(out.place.place_nom)
    # 
    # Roger = User.query.get(5): Roger est une classe, User
    # Roger.authorships : c'est une liste list[Authorship], équivalent de Authorship.query.filter(Authorship.authorships_user_id=Roger.user_id).all()
    # 
    user = db.relationship("User", back_populates="authorships")
    place = db.relationship("Place", back_populates="authorships")

class User(db.Model):
    user_id = db.Column(db.Integer, unique=True, nullable=False, primary_key=True, autoincrement=True)
    user_nom = db.Column(db.Text, nullable=False)
    user_login = db.Column(db.String(45), nullable=False, unique=True)
    user_email = db.Column(db.Text, nullable=False)
    user_password = db.Column(db.String(100), nullable=False)
    # Seul changement pour user: authorships est une propriété sous forme d'une liste
    authorships = db.relationship("Authorship", back_populates="user")
    
class Place(db.Model):
    place_id = db.Column(db.Integer, unique=True, nullable=False, primary_key=True, autoincrement=True)
    place_nom = db.Column(db.Text)
    place_description = db.Column(db.Text)
    place_longitude = db.Column(db.Float)
    place_latitude = db.Column(db.Float)
    place_type = db.Column(db.String(45))
    # Seul changement pour lieu
    authorships = db.relationship("Authorship", back_populates="place")


### Lecture du code

#### Construction "normale" du modèle

```python
class Authorship(db.Model):
    __tablename__ = "authorship"
    authorship_id = db.Column(db.Integer, nullable=True, autoincrement=True, primary_key=True)
    authorship_place_id = db.Column(db.Integer, db.ForeignKey('place.place_id'))
    authorship_user_id = db.Column(db.Integer, db.ForeignKey('user.user_id'))
    authorship_date = db.Column(db.DateTime, default=datetime.datetime.utcnow)
```

- On crée un `db.Model` `Authorship`
    - On en profite pour introduire `__tablename__` qui permet de spécifier un nom de table pour SQLAlchemy. Par défaut, SQLAlchemy utilise la version minuscule de la classe.
- On rajoute un champ `authorship_id` qui n'est pas très novateur
- On rajoute un champ `authorship_date` :
    - comme défaut, on utilise la fonction `datetime.utcnow`. Elle sera executée à chaque insertion
- On rajoute un champ `authorship_place_id`
    - Ce champ comporte une option `ForeignKey` qui lie au champ `place.place_id` qui permettra de les liens ensemble
    - On utilise les guillemets pour la `ForeignKey` qui correspondentà la syntaxe MySQL et non à la syntaxe python (`nomdetable.champ`)
- On rajoute un champ `authorship_user_id` construit en mirroir du champ précédent

##### Ce que l'on a appris en plus :

- `default` peut prendre une valeur par défaut, ou bien une fonction. Pour que la fonction soit exécutée à l'enregistrement, on fournit simplement le nom de la fonction.
- `ForeignKey` permet de lier un champ à un autre champ d'une autre table
- On peut utiliser `__tablename__` pour imposer un nom de table

#### Construction des relations en python

On distingue 4 nouveautés, dans les trois classes, introduites via `db.relationship()`.

```python
class Authorship(db.Model):
    # ...
    user = db.relationship("User", back_populates="authorships")
    place = db.relationship("Place", back_populates="authorships")

class User(db.Model):
    # ...
    authorships = db.relationship("Authorship", back_populates="user")
    
class Place(db.Model):
    # ...
    authorships = db.relationship("Authorship", back_populates="place")
```

**Réflexion:** Avant de lire les exemples, avez-vous une idée de leur effet ?

##### Exemple

In [11]:
# Exécuter une seule fois si possible

# On récupère un lieu
place_2 = Place.query.get(2)
# On récupère un utilisateur
user_1 = User.query.get(1)
# On crée un lien d'autorité comme ça, ou bien Authorship(authorship.user_id=1, authorship.place_id=2)
a_ecrit = Authorship(user=user_1, place=place_2) # on n'attend pas un entier à user, mais un objet de la classe user
# On enregistre
db.session.add(a_ecrit)
db.session.commit()

In [12]:
for authorship in Place.query.get(2).authorships: # ou in place_2.authorships
    print("{auteur} a écrit à {moment}".format(
        auteur=authorship.user.user_nom,
        moment=authorship.authorship_date
    ))

Administrator a écrit à 2020-02-24 12:43:02.648272
Administrator a écrit à 2020-02-24 12:45:20.496038
Administrator a écrit à 2020-02-24 12:47:15.437433


###  `db.relationship`
`db.relationship()` permet de construire des relations directes entre les objets et de naviguer entre eux. Ainsi, dans l'insertion, on n'utilise pas `place_id` ou `user_id`, mais bien `place` et `user`. 

La fonction `db.relationship` est tout à fait différente de `db.Column()` en ce qu'elle n'intervient pas sur la structure MySQL de vos classes, mais permet simplement de connecter vos différentes classes. Elle est construite sur deux paramètres:
- le premier paramètre correspond au nom de la classe Python qui est liée à la propriété
- le second paramètre (`back_populates`), qui est optionnel, correspond à la propriété dans la classe cible qui contient la même information en miroir. Ainsi `Authorship.user` lie vers `User` tandis que `User.authorships` lie vers `Authorship`.

En résumé, on a en syntaxe :

```python
class A(db.Model):
    propriete_de_relation = db.relationship(
        "NomDeLaClasseLiée",
        back_populates="a"
    )
class NomDeLaClasseLiée(db.Model):
    a = db.relationship(
        "A",
        back_populates="propriete_de_relation"
    )
```

![Schema](images/BackPopulate.png)

### Exercice

L'exercice se fait dans un notebook externe : [lien vers l'exercice](./Chapitre%2012%20-%20Jointures%20SQL%20et%20Update%20-%20Exercice%20Jointure.ipynb)


## Update, vous avez dit Update ?

En SQL
et en gestion de données, on parle généralement de CRUD : Create-Read-Update-Delete. Nous avons vu les deux premiers, le quatrième s'écrite simplement `Place.query.get(1).delete()` ce qui n'est pas très compliqué... Alors, comment corriger un enregistrement ?

La modification d'une ligne SQL se fait assez simplement :
1. On récupère l'objet
2. On modifie les propriétés que l'on souhaite modifier
3. On ajoute l'objet dans à la session de changement
4. On commit

Soit :

In [13]:

# 1.
place = Place.query.filter(Place.place_nom.like("%settlement%")).first()
print(place.place_nom)
# 2.
place.place_nom = "Lipara"
# 3. 
db.session.add(place)
# 4.
db.session.commit()

# Résultats:

print(Place.query.filter(Place.place_nom.like("%settlement%")).count())

Lipara (settlement)
0


#### Exercide

Ajouter un formulaire d'édition de lieu pour le projet Boatlas. 

1. On ne pourra pas changer l'id
2. La mise à jour rajoutera un lien d'autorité vers l'éditeur